In [1]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import random

In [58]:
class LoanOptions:
    def __init__(self,num_loans,B,T,L_T = [],L_B = [],L_R = [],L_M = [],min_p=1,max_p=100,min_t = 6,max_r=100):
        self.num_loans = num_loans
        self.B = B
        self.T = T
        self.L_T = L_T # number of loan payments for each loan
        self.L_B = L_B # max amount for each loan
        self.L_R = L_R # loan rate for each loan
        self.L_M = L_M # minimimum loan payment
        self.min_p = min_p
        self.max_p = max_p
        self.min_t = min_t
        self.max_r = max_r
    def randomly_generate_loans(self):
        self.L_T = [random.randint(self.min_t,self.T) 
                    for x in range(self.num_loans)]
        self.L_R = [ random.randint(1,self.max_r)/100 for x in range(self.num_loans)]
        self.L_B = [random.randint(1,self.B) for x in range(self.num_loans)]
        while(sum(self.L_B) < self.B):
            self.L_B = [random.randint(1,self.B) for x in range(self.num_loans)]
            
        self.L_M = [ random.randint(int(self.L_B[x]/self.L_T[x]),int(self.B/self.T))
                    if int(self.L_B[x]/self.L_T[x]) < int(self.B/self.T) 
                    else random.randint(int(self.B/self.T),int(self.L_B[x]/self.L_T[x]))
                   for x in range(self.num_loans)]     

In [ ]:
loan_data = LoanOptions(num_loans=5,B=5000,T=24)
loan_data.randomly_generate_loans()

In [ ]:
print(loan_data.L_B)
print(loan_data.L_R)
print(loan_data.L_T)
print(loan_data.L_M)

In [125]:
B = cp.Parameter(nonneg=True) # total amount we need to borrow
B.value = 5000
T = cp.Parameter(nonneg=True) # total number of months to pay off loan
T.value = 12
p = cp.Parameter(nonneg=True) # total monthly payment
p.value = int((B.value*(1.2))/T.value)

num_loans = 4
# generate our loan constants
loan_data = LoanOptions(num_loans=num_loans,B=B.value,T=T.value,max_r=60)
loan_data.randomly_generate_loans()
# convert object variables to parameters for more efficient use
# total number of payments to make for each loan
L_T = cp.Parameter(num_loans,nonneg=True)
L_T.value = np.array(loan_data.L_T)
# maximum loan amount for each loan
L_B = cp.Parameter(num_loans,nonneg=True)
L_B.value = np.array(loan_data.L_B)
# intrest rate for each loan
L_R = cp.Parameter(num_loans,nonneg=True)
L_R.value = np.array(loan_data.L_R)
# minimum payment for each loan
L_M = cp.Parameter(num_loans,nonneg=True)
#L_M.value = np.array(loan_data.L_M)
L_M.value = np.array([0]*num_loans)

# make our loan variables
total_payment = [cp.Variable(x,nonneg=True) for x in L_T.value]      
obj = cp.Minimize(cp.sum([cp.sum(x) for x in total_payment]))

# make our constraints
constraints = []

# total principal on all loans must equal borrowing needs
constraints.append(cp.sum(
    [cp.sum(loan_payments)/((1+L_R.value[i])**(L_T.value[i]/12))
     for i,loan_payments in enumerate(total_payment)])==B.value)

# total payments for each loan must be less than max borrow amount
npv = []
for i,loan_payments in enumerate(total_payment):
    npv.append(L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12))
    constraints.append(cp.sum(loan_payments)<=L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12))

# total payments for one loan must be less than T
for time_horizon in L_T:
    constraints.append(time_horizon <= T.value)   

# monthly payment on a loan is a minimum value m_i
for i,loan_payments in enumerate(total_payment):
    for payment in loan_payments:
        constraints.append(payment>= L_M.value[i])

# total monthly payment equal to p
for month in range(T.value):
    constraints.append(cp.sum([i[month] for i in total_payment if i.size > month])>=p)    

# solve
prob = cp.Problem(obj,constraints)
prob.solve()
print('solver: '+prob.solver_stats.solver_name)
print('minimal payment: ' + str(prob.value))
print('maximum borrowing amount: '+ str(L_B.value))
print('total months to pay each loan: '+ str(L_T.value))
print('minimum monthly payment for each loan: '+str(L_M.value))
print('loan interest rates: '+str(L_R.value))
print('Net Present Value of loans: '+str(npv))

for loan_payment in total_payment:
    print("loan payments: "+ str(loan_payment.value))

solver: ECOS
minimal payment: 6294.398079768032
maximum borrowing amount: [ 438 3543 4796 4484]
total months to pay each loan: [12  7  6 12]
minimum monthly payment for each loan: [0 0 0 0]
loan interest rates: [0.3  0.58 0.29 0.51]
Net Present Value of loans: [569.4, 4626.520097323997, 5447.208885291622, 6770.84]
loan payments: [3.14168987e-08 3.14168987e-08 3.14168987e-08 3.14168987e-08
 3.14168987e-08 3.14168987e-08 1.31088572e-07 1.13880000e+02
 1.13880000e+02 1.13880000e+02 1.13880000e+02 1.13880000e+02]
loan payments: [3.68190751e-08 3.68190751e-08 3.68190751e-08 3.68190751e-08
 3.68190751e-08 3.68190751e-08 5.00000000e+02]
loan payments: [549.06634626 549.06634626 549.06634626 549.06634626 549.06634626
 549.06634626]
loan payments: [1.53884625e-07 1.53884625e-07 1.53884625e-07 1.53884625e-07
 1.53884625e-07 1.53884625e-07 2.65713809e-07 3.86120000e+02
 3.86120000e+02 3.86120000e+02 3.86120000e+02 3.86120000e+02]


In [175]:
 def optimize_loans_scenario_1(b,t,required_payment,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment):
    B = cp.Parameter(nonneg=True) # total amount we need to borrow
    B.value = b
    T = cp.Parameter(nonneg=True) # total number of months to pay off loan
    T.value = t
    p = cp.Parameter(nonneg=True) # total monthly payment
    p.value = required_payment

    num_loans = len(max_loan_amounts)
    # generate our loan constants
    loan_data = LoanOptions(num_loans=num_loans,B=B.value,T=T.value,max_r=60)
    loan_data.randomly_generate_loans()
    # convert object variables to parameters for more efficient use
    # total number of payments to make for each loan
    L_T = cp.Parameter(num_loans,nonneg=True)
    L_T.value = np.array(loan_terms)
    # maximum loan amount for each loan
    L_B = cp.Parameter(num_loans,nonneg=True)
    L_B.value = np.array(max_loan_amounts)
    # intrest rate for each loan
    L_R = cp.Parameter(num_loans,nonneg=True)
    L_R.value = np.array(loan_rates)
    # minimum payment for each loan
    L_M = cp.Parameter(num_loans,nonneg=True)
    #L_M.value = np.array(loan_data.L_M)
    L_M.value = np.array(minimum_monthly_payment)#[0]*num_loans)

    # make our loan variables
    total_payment = [cp.Variable(x,nonneg=True) for x in L_T.value]      
    obj = cp.Minimize(cp.sum([cp.sum(x) for x in total_payment]))

    # make our constraints
    constraints = []

    # total principal on all loans must equal borrowing needs
    constraints.append(cp.sum(
        [cp.sum(loan_payments)/((1+L_R.value[i])**(L_T.value[i]/12))
         for i,loan_payments in enumerate(total_payment)])==B.value)

    # total payments for each loan must be less than max borrow amount
    npv = []
    for i,loan_payments in enumerate(total_payment):
        npv.append("{:0.2f}".format(L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12)))
        constraints.append(cp.sum(loan_payments)<=L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12))

    # total payments for one loan must be less than T
    for time_horizon in L_T:
        constraints.append(time_horizon <= T.value)   

    # monthly payment on a loan is a minimum value m_i
    for i,loan_payments in enumerate(total_payment):
        for payment in loan_payments:
            constraints.append(payment>= L_M.value[i])

    # total monthly payment >= to p
    for month in range(T.value):
        constraints.append(cp.sum([i[month] for i in total_payment if i.size > month])>=p)    
    
    # solve
    prob = cp.Problem(obj,constraints)
    prob.solve()
    #print('solver: '+prob.solver_stats.solver_name)
    print('minimal payment: ' + "{:0.2f}".format(prob.value))
    print('maximum borrowing amount: '+ str(L_B.value))
    print('total months to pay each loan: '+ str(L_T.value))
    print('minimum monthly payment for each loan: '+str(L_M.value))
    print('loan interest rates: '+str(L_R.value))
    print('Net Present Value of loans: '+str(npv))

    for i,loan_payment in enumerate(total_payment):
        print("loan payments for loan "+str(i+1)+ ": "+ "{:0.2f}".format(sum(loan_payment.value))) #{:0.2f}.format(sum(loan_payment.value))

In [180]:
 def optimize_loans_scenario_2(b,t,required_payment,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment):
    B = cp.Parameter(nonneg=True) # total amount we need to borrow
    B.value = b
    T = cp.Parameter(nonneg=True) # total number of months to pay off loan
    T.value = t
    p = cp.Parameter(nonneg=True) # total monthly payment
    p.value = required_payment

    num_loans = len(max_loan_amounts)
    # generate our loan constants
    loan_data = LoanOptions(num_loans=num_loans,B=B.value,T=T.value,max_r=60)
    loan_data.randomly_generate_loans()
    # convert object variables to parameters for more efficient use
    # total number of payments to make for each loan
    L_T = cp.Parameter(num_loans,nonneg=True)
    L_T.value = np.array(loan_terms)
    # maximum loan amount for each loan
    L_B = cp.Parameter(num_loans,nonneg=True)
    L_B.value = np.array(max_loan_amounts)
    # intrest rate for each loan
    L_R = cp.Parameter(num_loans,nonneg=True)
    L_R.value = np.array(loan_rates)
    # minimum payment for each loan
    L_M = cp.Parameter(num_loans,nonneg=True)
    #L_M.value = np.array(loan_data.L_M)
    L_M.value = np.array(minimum_monthly_payment)#[0]*num_loans)

    # make our loan variables
    total_payment = [cp.Variable(x,nonneg=True) for x in L_T.value]      
    obj = cp.Minimize(cp.sum([cp.sum(x) for x in total_payment]))

    # make our constraints
    constraints = []

    # total principal on all loans must equal borrowing needs
    constraints.append(cp.sum(
        [cp.sum(loan_payments)/((1+L_R.value[i])**(L_T.value[i]/12))
         for i,loan_payments in enumerate(total_payment)])==B.value)

    # total payments for each loan must be less than max borrow amount
    npv = []
    for i,loan_payments in enumerate(total_payment):
        npv.append("{:0.2f}".format(L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12)))
        constraints.append(cp.sum(loan_payments)<=L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12))

    # total payments for one loan must be less than T
    for time_horizon in L_T:
        constraints.append(time_horizon <= T.value)   

    # monthly payment on a loan is a minimum value m_i
    for i,loan_payments in enumerate(total_payment):
        for payment in loan_payments:
            constraints.append(payment>= L_M.value[i])

    # total monthly payment >= to p
    for month in range(T.value):
        constraints.append(cp.sum([i[month] for i in total_payment if i.size > month])<=p)    
    
    # solve
    prob = cp.Problem(obj,constraints)
    prob.solve()
    #print('solver: '+prob.solver_stats.solver_name)
    print('minimal payment: ' + "{:0.2f}".format(prob.value))
    print('maximum borrowing amount: '+ str(L_B.value))
    print('total months to pay each loan: '+ str(L_T.value))
    print('minimum monthly payment for each loan: '+str(L_M.value))
    print('loan interest rates: '+str(L_R.value))
    print('Net Present Value of loans: '+str(npv))

    for i,loan_payment in enumerate(total_payment):
        try:
            print("loan payments for loan "+str(i+1)+ ": "+ "{:0.2f}".format(sum(loan_payment.value))) 
        except:
            print('infeasible solution')

In [230]:
 def optimize_loans_scenario_3(b,t,required_payment,required_payment_interest,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment):
    B = cp.Parameter(nonneg=True) # total amount we need to borrow
    B.value = b
    T = cp.Parameter(nonneg=True) # total number of months to pay off loan
    T.value = t
    p = cp.Parameter(t,nonneg=True) # total monthly payment
    p.value = [0]*(t)
    for y in range(int(t/12)):
        for m in range(12):
            # want the monthy rent to compound interest every year
            p.value[y*12+m]=required_payment*(1+required_payment_interest)**(y+1)
    
    num_loans = len(max_loan_amounts)
    # generate our loan constants
    loan_data = LoanOptions(num_loans=num_loans,B=B.value,T=T.value,max_r=60)
    loan_data.randomly_generate_loans()
    # convert object variables to parameters for more efficient use
    # total number of payments to make for each loan
    L_T = cp.Parameter(num_loans,nonneg=True)
    L_T.value = np.array(loan_terms)
    # maximum loan amount for each loan
    L_B = cp.Parameter(num_loans,nonneg=True)
    L_B.value = np.array(max_loan_amounts)
    # intrest rate for each loan
    L_R = cp.Parameter(num_loans,nonneg=True)
    L_R.value = np.array(loan_rates)
    # minimum payment for each loan
    L_M = cp.Parameter(num_loans,nonneg=True)
    #L_M.value = np.array(loan_data.L_M)
    L_M.value = np.array(minimum_monthly_payment)#[0]*num_loans)

    # make our loan variables
    total_payment = [cp.Variable(x,nonneg=True) for x in L_T.value]      
    obj = cp.Minimize(cp.sum([cp.sum(x) for x in total_payment]))

    # make our constraints
    constraints = []

    # total principal on all loans must equal borrowing needs
    constraints.append(cp.sum(
        [cp.sum(loan_payments)/((1+L_R.value[i])**(L_T.value[i]/12))
         for i,loan_payments in enumerate(total_payment)])==B.value)

    # total payments for each loan must be less than max borrow amount
    npv = []
    for i,loan_payments in enumerate(total_payment):
        npv.append("{:0.2f}".format(L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12)))
        constraints.append(cp.sum(loan_payments)<=L_B.value[i]*(1+L_R.value[i])**(L_T.value[i]/12))

    # total payments for one loan must be less than T
    for time_horizon in L_T:
        constraints.append(time_horizon <= T.value)   

    # monthly payment on a loan is a minimum value m_i
    for i,loan_payments in enumerate(total_payment):
        for payment in loan_payments:
            constraints.append(payment>= L_M.value[i])

    # total monthly payment >= to p
    for month in range(T.value):
        constraints.append(cp.sum([i[month] for i in total_payment if i.size > month])<=p.value[month])    
    
    # solve
    prob = cp.Problem(obj,constraints)
    prob.solve()
    #print('solver: '+prob.solver_stats.solver_name)
    print('minimal payment: ' + "{:0.2f}".format(prob.value))
    print('maximum borrowing amount: '+ str(L_B.value))
    print('total months to pay each loan: '+ str(L_T.value))
    print('minimum monthly payment for each loan: '+str(L_M.value))
    print('loan interest rates: '+str(L_R.value))
    print('Net Present Value of loans: '+str(npv))

    for i,loan_payment in enumerate(total_payment):
        try:
            print("loan payments for loan "+str(i+1)+ ": "+ "{:0.2f}".format(sum(loan_payment.value))) 
        except:
            print('infeasible solution')



In [177]:
b = 500000
t = 12*15
p = 0
max_loan_amounts = [500000,500000]
loan_terms = [t,t]
loan_rates = [.02,.2]
minimum_monthly_payment = [0,0]
optimize_loans_scenario_1(b,t,p,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

minimal payment: 672934.17
maximum borrowing amount: [500000 500000]
total months to pay each loan: [180 180]
minimum monthly payment for each loan: [0 0]
loan interest rates: [0.02 0.2 ]
Net Present Value of loans: ['672934.17', '7703510.79']
loan payments for loan 1: 672934.17
loan payments for loan 2: 0.00


In [178]:
b = 500000
t = 12*15
p = 0
max_loan_amounts = [250000,500000]
loan_terms = [t,t]
loan_rates = [.02,.2]
minimum_monthly_payment = [0,0]
optimize_loans_scenario_1(b,t,p,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

minimal payment: 4188222.48
maximum borrowing amount: [250000 500000]
total months to pay each loan: [180 180]
minimum monthly payment for each loan: [0 0]
loan interest rates: [0.02 0.2 ]
Net Present Value of loans: ['336467.08', '7703510.79']
loan payments for loan 1: 336467.08
loan payments for loan 2: 3851755.39


In [179]:
b = 500000
t = 12*15
p = 0
max_loan_amounts = [500000,500000]
loan_terms = [int(t/2),t]
loan_rates = [.02,.02]
minimum_monthly_payment = [0,0]
optimize_loans_scenario_1(b,t,p,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

minimal payment: 580057.83
maximum borrowing amount: [500000 500000]
total months to pay each loan: [ 90 180]
minimum monthly payment for each loan: [0 0]
loan interest rates: [0.02 0.02]
Net Present Value of loans: ['580057.83', '672934.17']
loan payments for loan 1: 580057.83
loan payments for loan 2: 0.00


In [215]:
b = 100000
t = 12*15
annual_salary = 100000
p = (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.02]
minimum_monthly_payment = [0]
print('maximum monthly payment: '+ "{:0.2f}".format(p)+ '\n')

for i in range(1,5):
    max_loan_amounts = [b*i]
    optimize_loans_scenario_2(b*i,t,p,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)
    print('\n')

maximum monthly payment: 2500.00

minimal payment: 134586.83
maximum borrowing amount: [100000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.02]
Net Present Value of loans: ['134586.83']
loan payments for loan 1: 134586.83


minimal payment: 269173.67
maximum borrowing amount: [200000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.02]
Net Present Value of loans: ['269173.67']
loan payments for loan 1: 269173.67


minimal payment: 403760.50
maximum borrowing amount: [300000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.02]
Net Present Value of loans: ['403760.50']
loan payments for loan 1: 403760.50


minimal payment: inf
maximum borrowing amount: [400000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.02]
Net Present Value of loans: ['538347.34']
infeasible

In [276]:
# Median Bay area home and median bay area salary
b = 995800
t = 12*15
annual_salary = 119136
p = (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.04805]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum starting monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum starting monthly payment: 2978.40

minimal payment: inf
maximum borrowing amount: [995800]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.04805]
Net Present Value of loans: ['2013270.62']
infeasible solution


In [261]:
# Affordable home for median Bay Area salary
# with no compounding income it was around 275000
b = 328000
t = 12*15
annual_salary = 119136
p = (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.044321]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum starting monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum starting monthly payment: 2978.40

minimal payment: 628613.95
maximum borrowing amount: [328000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.044321]
Net Present Value of loans: ['628613.95']
loan payments for loan 1: 628613.95


In [300]:
# percent difference in median home value vs affordable home
(995800 - 328000)/995800

0.670616589676642

In [268]:
# affordable home price for median income in Bay Area vs median rent
# median rent is 2710 
# at the moment you won't beat the rent, you should rent instead of buy
b = 328000
t = 12*15
annual_salary = 119136
p = 2710#(annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.044321]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum starting monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum starting monthly payment: 2710.00

minimal payment: inf
maximum borrowing amount: [328000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.044321]
Net Present Value of loans: ['628613.95']
infeasible solution


In [292]:
# affordable home price for median income in Bay Area vs rent
# if the rent was 3000, then it would be cheaper to buy
b = 328000
t = 12*15
annual_salary = 119136
p = 3000#(annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.044321]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum starting monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum starting monthly payment: 3000.00

minimal payment: 628613.95
maximum borrowing amount: [328000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.044321]
Net Present Value of loans: ['628613.95']
loan payments for loan 1: 628613.95


In [277]:
# Median income and median home price in Mississippi
b = 161162
t = 12*15
annual_salary = 45792
p = (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.04424]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum monthly payment: 1144.80

minimal payment: inf
maximum borrowing amount: [161162]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.04424]
Net Present Value of loans: ['308508.78']
infeasible solution


In [287]:
# affordable home price for median income in Mississippi
# without compounding income a feasible price is 10500
b = 120000
t = 12*15
annual_salary = 45792
p = (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.04781]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum monthly payment: 1144.80

minimal payment: 241779.42
maximum borrowing amount: [120000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.04781]
Net Present Value of loans: ['241779.42']
loan payments for loan 1: 241779.42


In [299]:
# percent decrease in price to make the median home affordable
(161162 - 120000)/161162

0.2554076022883806

In [301]:
# affordable home price for median income in Mississippi vs median rent
# median rent 916
# at the moment you won't beat the rent, you should rent instead of buy
b = 120000
t = 12*15
annual_salary = 45792
p = 916# (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.04781]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum monthly payment: 916.00

minimal payment: inf
maximum borrowing amount: [120000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.04781]
Net Present Value of loans: ['241779.42']
infeasible solution


In [298]:
# affordable home price for median income in Mississippi vs  rent
# if the rent was this much, you'd be better off buying
b = 120000
t = 12*15
annual_salary = 45792
p = 1150# (annual_salary*.3)/12
loan_terms = [t]
loan_rates = [.04781]
minimum_monthly_payment = [0]
max_loan_amounts = [b]
print('maximum monthly payment: '+ "{:0.2f}".format(p)+ '\n')
optimize_loans_scenario_3(b,t,p,.02,max_loan_amounts,loan_terms,loan_rates,minimum_monthly_payment)

maximum monthly payment: 1150.00

minimal payment: 241779.42
maximum borrowing amount: [120000]
total months to pay each loan: [180]
minimum monthly payment for each loan: [0]
loan interest rates: [0.04781]
Net Present Value of loans: ['241779.42']
loan payments for loan 1: 241779.42
